In [1]:
import pandas as pd
import csv

labeled_corpus = pd.read_csv('C:/bg/multicampus/labeled_corpus.csv', encoding = 'utf-8', index_col = [0])
labeled_corpus = labeled_corpus[labeled_corpus['token_ngram'].notna()]

In [2]:
import numpy as np
from sklearn.pipeline import Pipeline

In [3]:
labeled_corpus

,token_ngram,label
0,"들/VV,국제/NNG,유가/NNG,크/VA,오르/VV,원화/NNG,엔/NNG,화간/...",-1
1,"위안/NNG,절상/NNG,원화/NNG,절상/NNG,가장/MAG,크/VA,추가/NNG...",1
2,"금융/NNG,불안사태/NNG,발생/NNG,하/VV,가운데/NNG,국제/NNG,환투기...",0
3,"금리갭/NNG,확대/NNG,되/XSV,가운데/NNG,남/VV,fed/NNG,금리/N...",1
4,"미/NNG,달러/NNG,엔/NNG,약세/NNG,지속/NNG,되/VV,보/VV,물/V...",0
5,"들/VV,달러/NNG,엔/NNG,유로/NNG,대해/VV,상당히/MAG,강세/NNG,...",1
6,"거의/MAG,예외/NNG,없이/MAG,원화/NNG,강세/NNG,현상/NNG,나타나/...",0
7,"자본/NNG,유입/NNG,대해서/VV,관대/NNG,반면/NNG,자본/NNG,유출/N...",1
8,"사례/NNG,보/VV,원화/NNG,환율/NNG,엔/NNG,환율/NNG,위안/NNG,...",1
9,"환율/NNG,하락/NNG,수급/NNG,사정/NNG,변화/NNG,금리/NNG,변동/N...",-1


In [4]:
labeled_corpus = labeled_corpus[labeled_corpus['label']!= 0]
labeled_corpus

,token_ngram,label
0,"들/VV,국제/NNG,유가/NNG,크/VA,오르/VV,원화/NNG,엔/NNG,화간/...",-1
1,"위안/NNG,절상/NNG,원화/NNG,절상/NNG,가장/MAG,크/VA,추가/NNG...",1
3,"금리갭/NNG,확대/NNG,되/XSV,가운데/NNG,남/VV,fed/NNG,금리/N...",1
5,"들/VV,달러/NNG,엔/NNG,유로/NNG,대해/VV,상당히/MAG,강세/NNG,...",1
7,"자본/NNG,유입/NNG,대해서/VV,관대/NNG,반면/NNG,자본/NNG,유출/N...",1
8,"사례/NNG,보/VV,원화/NNG,환율/NNG,엔/NNG,환율/NNG,위안/NNG,...",1
9,"환율/NNG,하락/NNG,수급/NNG,사정/NNG,변화/NNG,금리/NNG,변동/N...",-1
11,"세계/NNG,긴축/NNG,기조/NNG,영향/NNG,신흥시장/NNG,투자/NNG,축소...",1
12,"외평채/NNG,스프레드/NNG,이르/VA,미사일/NNG,발사/NNG,영향/NNG,대...",1
13,"외화대출/NNG,대폭/MAG,증가/NNG,하/XSV,은행/NNG,외화대출/NNG,재...",1


In [18]:
# labeled_corpus.to_csv('final_corpus_without_0.csv', encoding = 'utf-8')


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from tqdm import tqdm

In [6]:
def my_tokenizer(x):
    return x.split(",")

In [7]:
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1), min_df = 15, tokenizer = my_tokenizer)),
                      ('clf', MultinomialNB(alpha=0.001)), ])
vect = text_clf.named_steps['vect']
clf = text_clf.named_steps['clf']

In [11]:
#Bagging

avg_polarity_score = np.zeros((1, 18787))
accuracy = np.zeros(30)
for i in tqdm(range(30)):
    X_train, X_test, y_train, y_test = train_test_split(labeled_corpus['token_ngram'], labeled_corpus['label'], random_state = 0, train_size = 0.9, shuffle=True)
    text_clf.fit(X_train, y_train)
    clf = text_clf.named_steps['clf']
    posterior = clf.feature_log_prob_
    polarity_score = posterior[1]/posterior[0]
    avg_polarity_score += polarity_score
    pred = text_clf.predict(X_test)
    accuracy[i] = np.sum(pred == y_test)

avg_polarity_score = avg_polarity_score/30
print(avg_polarity_score)

100%|██████████| 30/30 [15:35<00:00, 31.18s/it][[0.94667116 0.9598659  0.99043297 ... 1.01128209 0.99340144 1.0446219 ]]



In [29]:
clf.classes_ 

array([-1,  1], dtype=int64)

In [12]:
vect = text_clf.named_steps['vect']

vect.get_feature_names()

['6자회담/nng',
 'a+등급/nng',
 'a-등급/nng',
 'aa+등급/nng',
 'aa-등급/nng',
 'aaa등급/nng',
 'aa등급/nng',
 'asp/nng',
 'a등급/nng',
 'bbb+등급/nng',
 'bbb-등급/nng',
 'bbb등급/nng',
 'bb등급/nng',
 'bei/nng',
 'bis/nng',
 'bps/nng',
 'bw/nng',
 'cac40/nng',
 'cb/nng',
 'cd/nng',
 'cd/nng;금리/nng;상승/nng',
 'cds/nng',
 'cp/nng',
 'crb/nng',
 'crs/nng',
 'db/nng',
 'dc/nng',
 'dps/nng',
 'dr/nng',
 'd의공포/nng',
 'eb/nng',
 'ebitda/nng',
 'ecb/nng',
 'ecb/nng;금리/nng;인상/nng',
 'els/nng',
 'eps/nng',
 'etf/nng',
 'etn/nng',
 'fcf/nng',
 'fed/nng',
 'fed/nng;금리/nng;인상/nng',
 'ff/nng',
 'ftse/nng',
 'ftse100/nng',
 'fx스왑/nng',
 'g7/nng',
 'g8/nng',
 'gdi/nng',
 'gdp/nng',
 'gdr/nng',
 'gni/nng',
 'icbm/nng',
 'ira/nng',
 'irs/nng',
 'ism/nng',
 'l자형/nng',
 'm&a/nng',
 'mmf/nng',
 'msci/nng',
 'ndf시장/nng',
 'oecd/nng',
 'ois스프레드/nng',
 'opec/nng',
 'p2p대출/nng',
 'pbr/nng',
 'pce/nng',
 'per/nng',
 'pf/nng',
 'pmi/nng',
 'qfii/nng',
 'rbc비율/nng',
 'rim/nng',
 'roa/nng',
 'roe/nng',
 'rp/nng',
 'rqfii/nng',
 's&p/nng',


In [40]:
avg_polarity_score.shape

(1, 18787)

In [34]:
len(vect.get_feature_names())

18787

In [13]:
polarity_scores = pd.DataFrame(list(avg_polarity_score), columns = vect.get_feature_names()).T
polarity_scores

,0
6자회담/nng,0.946671
a+등급/nng,0.959866
a-등급/nng,0.990433
aa+등급/nng,1.000245
aa-등급/nng,1.066672
aaa등급/nng,0.994153
aa등급/nng,1.018349
asp/nng,0.909473
a등급/nng,1.011986
bbb+등급/nng,0.981812


In [16]:
polarity_scores[polarity_scores[0].apply(lambda x: x > 1.3 or x < 0.7)]

,0
丙/nng,0.546383
甘/nng,1.784487
申年/nng,0.556876
鳶/nng,1.767705
걸/nng,1.806759
걸작/nng,1.784487
공유형모기지/nng,1.989147
교관/nng,1.784487
구민/nng,1.302410
구스타브/nng,0.508210
